In [1]:
pip install np_utils

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
import numpy as np
from tensorflow.keras.layers import Dense, Dropout, Input
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical

import tensorflow
tensorflow.random.set_seed(22)

# load MNIST dataset
(x_train, y_train), (x_test, y_test) = mnist.load_data()

# from sparse label to categorical
num_labels = len(np.unique(y_train))
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

# reshape and normalize input images
image_size = x_train.shape[1]
x_train = np.reshape(x_train,[-1, image_size, image_size, 1])
x_test = np.reshape(x_test,[-1, image_size, image_size, 1])
x_train = x_train.astype('float32') / 255
x_test = x_test.astype('float32') / 255

# network parameters
input_shape = (image_size, image_size, 1)
batch_size = 128
kernel_size = 3
filters = 64
dropout = 0.3

# use functional API to build cnn layers
inputs = Input(shape=input_shape)

y = Conv2D(filters=filters,
           kernel_size=kernel_size,
           activation='relu')(inputs)

y = MaxPooling2D()(y)

y = Conv2D(filters=filters,
           kernel_size=kernel_size,
           activation='relu')(y)

y = MaxPooling2D()(y)
# y = Conv2D(filters=filters,
#            kernel_size=kernel_size,
#            activation='relu')(y)
# image to vector before connecting to dense layer
y = Flatten()(y)
# dropout regularization
#y = Dropout(dropout)(y)
outputs = Dense(num_labels, activation='softmax')(y)



# build the model by supplying inputs/outputs
model = Model(inputs=inputs, outputs=outputs)
# network model in text
model.summary()
# Retrieve the config

# classifier loss, Adam optimizer, classifier accuracy
from tensorflow.keras.optimizers import Adam
# classifier loss, Adam optimizer, classifier accuracy
model.compile(loss='categorical_crossentropy',
              optimizer=Adam(learning_rate=0.0001),
              metrics=['accuracy'])

import time
t0 = time.time()
# train the model with input images and labels
model.fit(x_train,
          y_train,
          validation_data=(x_test, y_test),
          epochs=50,
          batch_size=batch_size)
print("Training time:", time.time()-t0)

# model accuracy on test dataset
score = model.evaluate(x_test,
                       y_test,
                       batch_size=batch_size,
                       verbose=0)
print("\nTest accuracy: %.1f%%" % (100.0 * score[1]))

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 28, 28, 1)]       0         
                                                                 
 conv2d (Conv2D)             (None, 26, 26, 64)        640       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 13, 13, 64)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 11, 11, 64)        36928     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 5, 5, 64)         0         
 2D)                                                             
                                                                 
 flatten (Flatten)           (None, 1600)              0     

Method_1 - Not all weights are printed

In [3]:
weight = model.get_weights()
np.savetxt('weight_method_1.csv' , weight , fmt='%s', delimiter=',')

/usr/local/lib/python3.8/dist-packages/numpy/lib/npyio.py:1378: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  X = np.asarray(X)


Method_2 - Printing weights to console

In [4]:
for i in model.layers:
        print("------")
        print(i, "\n", i.input_shape, "\n", i.output_shape, "\n", i.get_weights())

------
 [(None, 28, 28, 1)] 
 [(None, 28, 28, 1)] 
 []
------
 (None, 28, 28, 1) 
 (None, 26, 26, 64) 
 [array([[[[-1.80219978e-01,  1.58700526e-01,  1.40067384e-01,
           1.01181969e-01,  2.71935239e-02,  1.43903732e-01,
           1.46139145e-01,  2.02964574e-01,  5.35398610e-02,
           1.75139830e-01,  1.84737548e-01,  2.05435917e-01,
           1.78475052e-01,  1.41623184e-01,  1.22648790e-01,
          -3.28015499e-02,  2.26218570e-02,  1.56232137e-02,
          -2.25300297e-01,  3.93555835e-02,  1.23589724e-01,
           2.95741912e-02,  1.34825945e-01, -1.12767316e-01,
           1.93588540e-01,  9.14105922e-02,  2.10980967e-01,
           2.44016899e-03,  1.30973250e-01,  3.08532625e-01,
           2.49243796e-01,  4.91356030e-02,  2.10800886e-01,
           9.82838124e-02, -2.32862309e-01,  1.05640151e-01,
           1.28863111e-01,  4.14711721e-02, -6.35426119e-02,
          -2.69169509e-02,  1.45494118e-01,  1.75936207e-01,
           6.26810044e-02,  1.53128579e-0

Saving the model into .h5 

In [5]:
# serialize weights to HDF5
model.save_weights("model.h5")
print("Saved model to disk")

Saved model to disk


In [10]:
wt = open('weights.csv', 'w')
bs = open('biases.csv', 'w')
for idx,i in enumerate(model.layers):
  if(isinstance(i, Conv2D) or isinstance(i, Dense)):
    weights = i.get_weights()[0]
    biases = i.get_weights()[1]
    weights = weights.flatten()
    biases = biases.flatten()
    print(weights.shape, biases.shape)
    wt.write(','.join(map(str,weights.tolist()))+"\n")
    bs.write(','.join(map(str,biases.tolist()))+"\n")
    np.savetxt("weight" + str(idx)+".csv" , weights , fmt='%s', delimiter=',')
    np.savetxt("bias" + str(idx) +".csv" , biases , fmt='%s', delimiter=',')

(576,) (64,)
(36864,) (64,)
(16000,) (10,)
